In [ ]:
%matplotlib inline
import numpy as np
import scipy.io
import scipy.ndimage
import scipy.signal
import matplotlib.pyplot as plt
import matplotlib.animation
import cmasher as cmr
import os

folder=r"J:\ctgroup\Edward\DATA\VMI\20250303\Propylene Oxide 2W"
files=[os.path.join(folder,f) for f in os.listdir(folder) if f.endswith('.mat')]
angles=[float(f.split("\\")[-1].split('_')[0]) for f in files]

angles,files=zip(*sorted(zip(angles,files)))
def load_file(file, smoothed=True, plot=False, py_shift=0):
    data=scipy.io.loadmat(file,squeeze_me=True)
    px,py,pz=data['px'],data['py']+py_shift,data['pz']
    h=np.histogram(np.sqrt(px**2+py**2+pz**2),bins=500,range=(0,1))
    
    hist,edges=h[0],h[1]
    if smoothed:
        hist=scipy.signal.savgol_filter(hist,51,3)
    hist=hist/np.max(hist)
    
    peaks,properties=scipy.signal.find_peaks(hist,prominence=0.07)
    peak_widths=scipy.signal.peak_widths(hist,peaks,rel_height=0.2)[0]
    
    ring_vals=np.array([edges[int(peaks[i])] for i in range(len(peaks))])
    ring_widths=np.array([edges[int(peak_widths[i])] for i in range(len(peaks))])

    if plot:
        plt.figure(file)
        plt.plot(edges[:-1],hist)
        plt.plot(edges[peaks],hist[peaks],'x')
        [plt.plot([ring_vals[i]-ring_widths[i]*2,ring_vals[i]+ring_widths[i]*2],[0.5,0.5]) for i in range(len(ring_vals))]
    pr=np.sqrt(px**2+py**2+pz**2)
    masks=[(pr>ring_vals[i]-ring_widths[i]*2) & (pr<ring_vals[i]+ring_widths[i]*2) for i in range(len(ring_vals))]
    reasonable=np.abs(pz)<np.max(px)

    rings=[(px[masks[i]],py[masks[i]],pz[masks[i]]) for i in range(len(ring_vals))]
    return {
        'px':px[reasonable],
        'py':py[reasonable],
        'pz':pz[reasonable],
        'rings':rings,
        'ring_vals':ring_vals,
        'ring_widths':ring_widths
    }
data=[load_file(f,plot=False, py_shift=-8.35e-4-5.4146447e-05) for f in files]

plt.show()

In [ ]:
%matplotlib inline
import skimage

datum=data[np.argmin(np.abs(angles))]
sl=np.array(datum['pz']<0.01)
window=np.hypot(datum['py'],datum['px'])<0.6
sl=sl & window
yz_hist,ye,ze=np.histogram2d(datum['py'][sl],datum['pz'][sl],bins=256)
plt.figure()
plt.imshow(yz_hist.T,cmap=cmr.rainforest, extent=[-0.6,0.6,-0.6,0.6], origin='lower')
median=np.median(datum['py'][sl&(datum['px']<0.01)]), np.median(datum['px'][sl])
plt.scatter(0,0)
plt.scatter(*median)
print(median)
# signal=yz_hist
# signal=skimage.filters.difference_of_gaussians(signal,low_sigma=5,high_sigma=10)
# structure=skimage.feature.structure_tensor(signal)
# structure_evals=skimage.feature.structure_tensor_eigenvalues(structure)
# plt.figure()
# plt.imshow(structure_evals[0])
plt.show()

plt.figure()

fbas=[(sum(d['rings'][0][1]>0.01)-sum(d['rings'][0][1]<-0.01))/len(d['rings'][0][1]) for d in data]
plt.plot(angles,fbas)
plt.show()

In [ ]:
n=256
thetav,phiv=np.linspace(0,np.pi,n),np.linspace(-np.pi,np.pi,n*2)
theta,phi=np.meshgrid(thetav,phiv)
dtheta,dphi=np.pi/n,2*np.pi/n/2
domega=np.sin(theta)*dtheta*dphi
lmax=19
Y={(l,m): scipy.special.sph_harm(m,l,phi,theta) for l in range(lmax+1) for m in range(-l,l+1)}
Y_real={(l,m): np.real(
    int(m<0)*np.sqrt(1/2)*(Y[(l,-m)]-1j*Y[(l,m)])+
    int(m==0)*Y[(l,0)]+
    int(m>0)*np.sqrt(1/2)*(Y[(l,m)]+1j*Y[(l,-m)])*(-1)**m

)
    for l in range(lmax+1) for m in range(-l,l+1)
}
Y=Y_real

def get_beta(data,ring_number,Y,lmax,smoothing=3, symmetrize=False, n=1, k=0):
    ring=data['rings'][ring_number]
    px,py,pz=ring
    px,py,pz=px[k::n],py[k::n],pz[k::n]
    if symmetrize:
        px,py,pz=np.vstack([px,-px]),np.vstack([py,py]),np.vstack([pz,-pz])
        px,py,pz=px.flatten(),py.flatten(),pz.flatten()
        
    pr=np.sqrt(px**2+py**2+pz**2)
    ptheta=np.arccos(py/pr)
    pphi=np.arctan2(pz,px)
    
    hist=np.histogram2d(pphi,ptheta,bins=(len(phiv),len(thetav)),density=True)[0]
    if smoothing>0:
        hist=scipy.ndimage.gaussian_filter(hist,smoothing)
    beta={(l,m): np.sum(hist*Y[(l,m)]*domega) for l in range(lmax+1) for m in range(-l,l+1)}
    return beta

betas={(angle,ring): get_beta(datum,ring,Y_real,lmax) for angle,datum in zip(angles,data) for ring in range(len(datum['rings']))}

In [ ]:
plt.close("all")

# fig,ax=plt.subplots(lmax+1,2*lmax+1,figsize=(10,4))
# for l in range(lmax+1):
#     for m in range(-l,l+1):
#         ax[l,m+lmax].imshow(Y[(l,m)].T,cmap=cmr.fusion)
#         ax[l,m+lmax].set_title(f"l={l},m={m}")
# 
# [axi.set_axis_off() for axi in ax.ravel() if axi]
# plt.tight_layout()

fig,ax=plt.subplots(2,3,figsize=(10,6))
fnum=2
datum=data[fnum]
px,py,pz=datum["rings"][0]
px,py,pz=np.vstack([px,-px]).flatten(),np.vstack([py,py]).flatten(),np.vstack([pz,-pz]).flatten()
ax[0,0].hist2d(px,py,bins=256)
ax[0,1].hist2d(px,pz,bins=256)
ax[0,2].hist2d(py,pz,bins=256)
px,py,pz=data[-(fnum+1)]["rings"][0]
px,py,pz=np.vstack([px,-px]).flatten(),np.vstack([py,py]).flatten(),np.vstack([pz,-pz]).flatten()
ax[1,0].hist2d(px,py,bins=256)
ax[1,1].hist2d(px,pz,bins=256)
ax[1,2].hist2d(py,pz,bins=256)
[axi.set_axis_off() for axi in ax.ravel() if axi]
plt.tight_layout()

fig,ax=plt.subplots(2,3,figsize=(10,6))
r=((-0.5,0.5),(-0.5,0.5))
px,py,pz=data[fnum]["px"],data[fnum]["py"],data[fnum]["pz"]
ax[0,0].hist2d(px,py,bins=256,range=r)
ax[0,1].hist2d(px,pz,bins=256,range=r)
ax[0,2].hist2d(py,pz,bins=256,range=r)
px,py,pz=data[-(fnum+1)]["px"],data[-(fnum+1)]["py"],data[-(fnum+1)]["pz"]
ax[1,0].hist2d(px,py,bins=256,range=r)
ax[1,1].hist2d(px,pz,bins=256,range=r)
ax[1,2].hist2d(py,pz,bins=256,range=r)
[axi.set_axis_off() for axi in ax.ravel() if axi]
plt.tight_layout()
plt.show()

In [ ]:
ring=0
%matplotlib inline
lm=5
# angle_corr=[angle-10.5 for angle in angles]
angle_corr=angles
angle_lims=(-60,60)
sampled_betas=[betas[(angle,ring)] for angle in angles]
plt.close("all")
plt.figure(figsize=(10,6))
plt.subplot(221)
plt.title("Even l, Even m \n(Sym in plane, Sym Forwards/Backwards)")
# [plt.plot(angle_corr,[b[(l,m)] for b in sampled_betas],label=f"l={l},m={m}") for l in range(0,lm+1,2) for m in range(-l,l+1,2)]
[plt.plot(angle_corr,[b[(l,m)]+b[(l,-m)] for b in sampled_betas],label=f"l={l},|m|={m}") for l in range(0,lm+1,2) for m in range(0,l+1,2)]
plt.xlim(angle_lims)
plt.grid()
plt.legend()
plt.xlabel("Angle (deg)")
plt.subplot(222)
plt.title("Even l, Odd m \n(Antisym in plane, Antisym Forwards/Backwards)")
# [plt.plot(angle_corr,[b[(l,m)] for b in sampled_betas],label=f"l={l},m={m}") for l in range(0,lm+1,2) for m in range(-l+1,l,2)]
[plt.plot(angle_corr,[b[(l,m)]+b[(l,-m)] for b in sampled_betas],label=f"l={l},|m|={m}") for l in range(0,lm+1,2) for m in range(1,l+1,2)]
plt.xlim(angle_lims)
plt.grid()
plt.legend()
plt.xlabel("Angle (deg)")
plt.subplot(223)
plt.title("Odd l, Odd m \n(Antisym in plane, Sym Forwards/Backwards)")
# [plt.plot(angle_corr,[b[(l,m)] for b in sampled_betas],label=f"l={l},m={m}") for l in range(1,lm+1,2) for m in range(-l,l+1,2)]
[plt.plot(angle_corr,[b[(l,m)]+b[(l,-m)] for b in sampled_betas],label=f"l={l},|m|={m}") for l in range(1,lm+1,2) for m in range(1,l+1,2)]
plt.xlim(angle_lims)
plt.legend()
plt.xlabel("Angle (deg)")
plt.grid()
plt.subplot(224)
plt.title("Odd l, Even m \n(Sym in plane, Antisym Forwards/Backwards)")
# [plt.plot(angle_corr,[b[(l,m)] for b in sampled_betas],label=f"l={l},m={m}") for l in range(1,lm+1,2) for m in range(-l+1,l,2)]
[plt.plot(angle_corr,[b[(l,m)]+b[(l,-m)] for b in sampled_betas],label=f"l={l},|m|={m}") for l in range(1,lm+1,2) for m in range(0,l+1,2)]
plt.xlim(angle_lims)
plt.xlabel("Angle (deg)")
plt.grid()
plt.legend()
plt.suptitle(f"Beta Parameters, Ring {ring+1}")
plt.tight_layout()
plt.figure(figsize=(10,6))
plt.title("Odd l, m=0, Deviations from linear")
# [plt.plot(angle_corr,[b[(l,m)] for b in sampled_betas],label=f"l={l},m={m}") for l in range(1,lm+1,2) for m in range(-l+1,l,2)]
[
    plt.plot(angle_corr,
          [(b[(l,0)]-sampled_betas[len(angles)//2][(l,0)]) for b in sampled_betas],'o-',
          label=f"l={l}, m=0, Total change={sampled_betas[-1][(l,0)]-sampled_betas[0][(l,0)]:.4f}")
    for l in range(1,lm+1,2)
]
plt.xlim(angle_lims)
plt.xlabel("Angle (deg)")
plt.legend()
plt.show()

In [ ]:
even=[[b[(l,m)]+b[(l,-m)] for b in sampled_betas] for l in range(0,lm+1,2) for m in [0]]
angle_ss=np.linspace(angles[0],angles[-1],10000)
even_interp=[np.interp(angle_ss,angles,even[i]) for i in range(len(even))]
even_interp=[scipy.signal.savgol_filter(e,5,3) for e in even_interp]

plt.figure(figsize=(10,6))
plt.title("d/dtheta Even l, Even m \n(Sym in plane, Sym Forwards/Backwards)")
[plt.plot(angle_ss,np.gradient(even_interp[i],angle_ss)) for i in range(len(even))]

mask=np.argwhere(np.abs(angles)<20).flatten()
angles_np = np.array(angles)[mask]
even_np=[np.array(e)[mask] for e in even]
plt.figure(figsize=(10,6))
plt.title("Even l, Even m \n(Sym in plane, Sym Forwards/Backwards)")
[plt.plot(angles_np,even_np[i]) for i in range(len(even))]

#quadratic fit
def quad(x,a,b,c):
    return a*x**2+b*x+c
from scipy.optimize import curve_fit
popt=[curve_fit(quad,angles_np,even_np[i])[0] for i in range(len(even))]
even_fit=[quad(angles_np,*p) for p in popt]
[plt.plot(angles_np,even_fit[i]) for i in range(len(even))]
[plt.axvline(-p[1]/(2*p[0]),color='k',linestyle='--') for p in popt]
print([-p[1]/(2*p[0]) for p in popt])

mean_0=np.mean([-p[1]/(2*p[0]) for p in popt])
print(mean_0)

plt.figure(figsize=(10,6))
plt.title(f"Zero Angle: {mean_0}")
[plt.plot(angle_ss,even_interp[i]) for i in range(len(even))]
plt.axvline(mean_0,color='k',linestyle='--')
plt.axvline(mean_0-45,color='k',linestyle='--')
plt.axvline(mean_0+45,color='k',linestyle='--')
print(mean_0-45,mean_0+45)
plt.show()

In [ ]:
# index=0
ring=0
%matplotlib inline
plt.close("all")
for index in range(15):
    plt.figure(figsize=(15,5))
    plt.subplot(221)
    reconstruction=np.sum([betas[(angles[index],ring)][(l,m)]*Y[(l,m)] for l in range(lmax+1) for m in range(-l,l+1)],axis=0)
    plt.imshow(reconstruction.T,cmap=cmr.fusion)
    plt.gca().set_axis_off()
    plt.gca().set_aspect('auto')
    plt.colorbar()

    plt.subplot(223)
    odd_reconstruction=np.sum([betas[(angles[index],ring)][(l,m)]*Y[(l,m)] for l in range(1,lmax+1,2) for m in range(-l+1,l,2)],axis=0)
    plt.imshow(odd_reconstruction.T,cmap=cmr.fusion)
    plt.gca().set_axis_off()
    plt.gca().set_aspect('auto')
    plt.colorbar()


    plt.subplot(222)
    px,py,pz=data[index]["rings"][ring]
    pr=np.sqrt(px**2+py**2+pz**2)
    ptheta=np.arccos(py/pr)
    pphi=np.arctan2(pz,px)
    hist=np.histogram2d(pphi,ptheta,bins=(128,128),density=True)[0]
    plt.imshow(hist.T,cmap=cmr.fusion)
    plt.gca().set_axis_off()
    plt.gca().set_aspect('auto')
    plt.colorbar()

    plt.subplot(224)
    odd_data= hist - hist[:,::-1]
    plt.imshow(odd_data.T,cmap=cmr.fusion)

    plt.gca().set_axis_off()
    plt.gca().set_aspect('auto')
    plt.colorbar()
plt.show()

In [ ]:
n_subsamples=5
lmax=5
sub_betas={
    (angle,ring,n): get_beta(datum,ring,Y,lmax,n=n_subsamples,k=n)
    for datum,angle in zip(data,angles) for n in range(n_subsamples) for ring in [0]
}

sub_betas_mean={
    (angle,ring):{
        (l,m): np.mean([sub_betas[(angle,ring,n)][(l,m)] for n in range(n_subsamples)],axis=0)
        for l in range(lmax+1) for m in range(-l,l+1)
    }
    for angle in angles for ring in [0]
}
sub_betas_std={
    (angle,ring):{
        (l,m): np.std([sub_betas[(angle,ring,n)][(l,m)] for n in range(n_subsamples)],axis=0)
        for l in range(lmax+1) for m in range(-l,l+1)
    }
    for angle in angles for ring in [0]
}


In [ ]:
plt.close("all")
ring=0
%matplotlib nbagg
plt.figure(figsize=(15,10))
plt.subplot(221)
plt.title("Even l, Even m \n(Sym in plane, Sym Forwards/Backwards)")
[
    plt.errorbar(angles,[sub_betas_mean[(angle,ring)][(l,m)] for angle in angles],
                    [sub_betas_std[(angle,ring)][(l,m)] for angle in angles],
                    label=f"l={l},m={m}")
    for l in range(0,lm+1,2) for m in range(0,l+1,2)
]
plt.xlim(angle_lims)
plt.grid()
plt.legend()
plt.xlabel("Angle (deg)")

plt.subplot(222)
plt.title("Even l, Odd m \n(Antisym in plane, Antisym Forwards/Backwards)")
[
    plt.errorbar(angles,[sub_betas_mean[(angle,ring)][(l,m)] for angle in angles],
                    [sub_betas_std[(angle,ring)][(l,m)] for angle in angles],
                    label=f"l={l},m={m}")
    for l in range(0,lm+1,2) for m in range(1,l+1,2)
]
plt.xlim(angle_lims)
plt.grid()
plt.legend()

plt.subplot(223)
plt.title("Odd l, Odd m \n(Antisym in plane, Sym Forwards/Backwards)")
[
    plt.errorbar(angles,[sub_betas_mean[(angle,ring)][(l,m)] for angle in angles],
                    [sub_betas_std[(angle,ring)][(l,m)] for angle in angles],
                    label=f"l={l},m={m}")
    for l in range(1,lm+1,2) for m in range(1,l+1,2)
]
plt.xlim(angle_lims)
plt.grid()
plt.legend()

plt.subplot(224)
plt.title("Odd l, Even m \n(Sym in plane, Antisym Forwards/Backwards)")
[
    plt.errorbar(angles,[sub_betas_mean[(angle,ring)][(l,m)] for angle in angles],
                    [sub_betas_std[(angle,ring)][(l,m)] for angle in angles],
                    label=f"l={l},m={m}")
    for l in range(1,lm+1,2) for m in range(0,l+1,2)
]
plt.xlim(angle_lims)
plt.grid()
plt.legend()
plt.show()